## Normal (aka Non-Win) Beta Calculation

### Import Packages

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
import statsmodels.api as sm
import statistics
import sys
sys.path.insert(0, "../")
import util
import multiprocessing as mp
from  timeit import default_timer as timer
from importlib import reload
reload(util)

<module 'util' from '/Users/natejensen/Desktop/ASAM/Low Beta/Code/util.py'>

### Read in util.py Function Bank

In [2]:
from IPython.lib.backgroundjobs import BackgroundJobFunc

with open('util.py') as code:
    job = BackgroundJobFunc(exec, code.read())

result = job.run()

### Set Local Macro Variables

In [4]:
# Num of top market equity to keep
numstocks=3000

# Trailing window
window=1

### Import Data

#### Daily CRSP File

In [5]:
crsp_d = pd.read_csv('qcrspdsf_raw.csv.gz', compression='gzip', usecols=lambda x: x.lower())

crsp_d.columns = crsp_d.columns.str.lower()
crsp_d = crsp_d[['permno', 'permco', 'date', 'ret', 'retx', 'shrout', 'prc']]
crsp_d[['permno','permco']] = crsp_d[['permno','permco']].astype(int)
crsp_d['ret'] = pd.to_numeric(crsp_d['ret'], errors='coerce')
crsp_d['retx'] = pd.to_numeric(crsp_d['retx'], errors='coerce')

crsp_d['date'] = pd.to_datetime(crsp_d['date'])

crsp_d.head()

/Users/natejensen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,permno,permco,date,ret,retx,shrout,prc
0,10001,7953,1990-01-02,0.000000,0.000000,1022.0,10.1250
1,10001,7953,1990-01-03,-0.012346,-0.012346,1022.0,10.0000
2,10001,7953,1990-01-04,0.000000,0.000000,1022.0,10.0000
3,10001,7953,1990-01-05,0.006250,0.006250,1022.0,-10.0625
4,10001,7953,1990-01-08,0.006211,0.006211,1022.0,10.1250


In [6]:
crsp_d.dtypes

permno             int64
permco             int64
date      datetime64[ns]
ret              float64
retx             float64
shrout           float64
prc              float64
dtype: object

#### CRSP Daily Stock Event - Delisting

In [7]:
qcrspdse_raw = pd.read_csv('qcrspdse_raw.csv.gz', compression='gzip')

qcrspdse_raw = qcrspdse_raw[['permno', 'shrcd', 'exchcd', 'namedt', 'nameendt']]

qcrspdse_raw[['permno','shrcd','exchcd']] = qcrspdse_raw[['permno','shrcd','exchcd']].astype(int)
qcrspdse_raw = qcrspdse_raw[qcrspdse_raw['exchcd'].isin([1, 2, 3])]
qcrspdse_raw = qcrspdse_raw[qcrspdse_raw['shrcd'].isin([10, 11])]

qcrspdse_raw['namedt'] = pd.to_datetime(qcrspdse_raw['namedt'])
qcrspdse_raw['nameendt'] = pd.to_datetime(qcrspdse_raw['nameendt'])

qcrspdse_raw.head()

,permno,shrcd,exchcd,namedt,nameendt
0,10000,10,3,1986-01-07,1986-12-03
1,10000,10,3,1986-12-04,1987-03-09
2,10000,10,3,1987-03-10,1987-06-11
3,10001,11,3,1986-01-09,1993-11-21
4,10001,11,3,1993-11-22,2004-06-09


##### Join monthly stock data with name history

In [8]:
crsp_d = crsp_d.merge(qcrspdse_raw, on='permno', how='left')

crsp_d = crsp_d[(crsp_d.namedt <= crsp_d.date) & (crsp_d.date <= crsp_d.nameendt)]

crsp_d.head()

,permno,permco,date,ret,retx,shrout,prc,shrcd,exchcd,namedt,nameendt
0,10001,7953,1990-01-02,0.000000,0.000000,1022.0,10.1250,11.0,3.0,1986-01-09,1993-11-21
10,10001,7953,1990-01-03,-0.012346,-0.012346,1022.0,10.0000,11.0,3.0,1986-01-09,1993-11-21
20,10001,7953,1990-01-04,0.000000,0.000000,1022.0,10.0000,11.0,3.0,1986-01-09,1993-11-21
30,10001,7953,1990-01-05,0.006250,0.006250,1022.0,-10.0625,11.0,3.0,1986-01-09,1993-11-21
40,10001,7953,1990-01-08,0.006211,0.006211,1022.0,10.1250,11.0,3.0,1986-01-09,1993-11-21


#### S&P 500 Daily Index

In [9]:
crsp_sp = pd.read_csv('qcrsp500d_raw.csv.gz', compression='gzip')

crsp_sp = crsp_sp.rename(columns={'caldt':'date'})
crsp_sp['date'] = pd.to_datetime(crsp_sp['date'])

crsp_sp.head()

,date,vwretd,vwretx,ewretd,ewretx,totval,totcnt,usdval,usdcnt,spindx,sprtrn
0,1925-12-31,NaN,NaN,NaN,NaN,15236829.5,89.0,NaN,NaN,NaN,NaN
1,1926-01-02,0.004297,0.004297,0.002941,0.002941,15319686.5,89.0,15236829.5,79.0,NaN,NaN
2,1926-01-04,-0.001357,-0.001357,0.001036,0.001036,15298901.2,89.0,15319686.5,80.0,NaN,NaN
3,1926-01-05,-0.004603,-0.004653,-0.005856,-0.006063,15227711.5,89.0,15298901.2,80.0,NaN,NaN
4,1926-01-06,0.000537,0.000537,0.000888,0.000888,15235889.2,89.0,15227711.5,80.0,NaN,NaN


#### CRSP Daily Stock Event - Delisting

In [10]:
dlret = pd.read_csv('qdlretd_raw.csv.gz', compression='gzip')

dlret = dlret[['permno', 'dlret', 'dlstdt']]

dlret[['permno']] = dlret[['permno']].astype(int)

dlret['date'] = pd.to_datetime(dlret['dlstdt'])

dlret.head()

,permno,dlret,dlstdt,date
0,10000,0.000000,1987-06-11,1987-06-11
1,10001,0.000000,2017-08-03,2017-08-03
2,10002,0.010906,2013-02-15,2013-02-15
3,10003,-0.003648,1995-12-15,1995-12-15
4,10004,NaN,1986-01-17,1986-01-17


##### Format Dates

In [11]:
crsp_d = util.jdate(crsp_d)
crsp_sp = util.jdate(crsp_sp)
dlret = util.jdate(dlret)

### Trim Stocks

#### Import Saved Permno from Program 2a (aka Investible Universe)

In [12]:
permlist = pd.read_csv('qpermlist.csv.gz', compression='gzip')

permlist['jdate'] = pd.to_datetime(permlist['jdate'])

In [13]:
permlist['month']=permlist['jdate'].dt.month
permlist['year']=permlist['jdate'].dt.year

# Only keep permno that show up in top 3000 in December of that year,
permlist=permlist[permlist['month']==12]

In [14]:
print(permlist)

        permno      jdate  month  year
0        10001 2011-12-31     12  2011
1        10001 2012-12-31     12  2012
2        10001 2014-12-31     12  2014
3        10001 2016-12-31     12  2016
4        10002 2002-12-31     12  2002
...        ...        ...    ...   ...
199398   93436 2016-12-31     12  2016
199399   93436 2017-12-31     12  2017
199400   93436 2018-12-31     12  2018
199401   93436 2019-12-31     12  2019
199402   93436 2020-12-31     12  2020

[199403 rows x 4 columns]


In [15]:
# Used for trimming crsp_d
permshort=(permlist['permno'].drop_duplicates())

print(permshort)

0         10001
4         10002
10        10003
12        10006
71        10008
          ...  
199384    93430
199385    93433
199386    93434
199391    93435
199392    93436
Name: permno, Length: 17384, dtype: int64


In [16]:
# Trim to last 30ish years
crsp_d['year'] = crsp_d['jdate'].dt.year
crsp_d = crsp_d[crsp_d['year']>=1990]
permlist['year'] = permlist['jdate'].dt.year
permlist = permlist[permlist['year']>=1990]

crsp_d.head()

,permno,permco,ret,retx,shrout,prc,shrcd,exchcd,namedt,nameendt,jdate,year
0,10001,7953,0.000000,0.000000,1022.0,10.1250,11.0,3.0,1986-01-09,1993-11-21,1990-01-02,1990
10,10001,7953,-0.012346,-0.012346,1022.0,10.0000,11.0,3.0,1986-01-09,1993-11-21,1990-01-03,1990
20,10001,7953,0.000000,0.000000,1022.0,10.0000,11.0,3.0,1986-01-09,1993-11-21,1990-01-04,1990
30,10001,7953,0.006250,0.006250,1022.0,-10.0625,11.0,3.0,1986-01-09,1993-11-21,1990-01-05,1990
40,10001,7953,0.006211,0.006211,1022.0,10.1250,11.0,3.0,1986-01-09,1993-11-21,1990-01-08,1990


### Combine Returns

In [17]:
crsp = util.comebineRet(crsp_d,dlret)

crsp.head()

,permno,permco,ret,retx,shrout,prc,shrcd,exchcd,namedt,nameendt,jdate,year,dlret,dlstdt,retadj,retxadj
0,10001,7953,0.000000,0.000000,1022.0,10.1250,11.0,3.0,1986-01-09,1993-11-21,1990-01-02,1990,0.0,NaN,0.000000,0.000000
1,10001,7953,-0.012346,-0.012346,1022.0,10.0000,11.0,3.0,1986-01-09,1993-11-21,1990-01-03,1990,0.0,NaN,-0.012346,-0.012346
2,10001,7953,0.000000,0.000000,1022.0,10.0000,11.0,3.0,1986-01-09,1993-11-21,1990-01-04,1990,0.0,NaN,0.000000,0.000000
3,10001,7953,0.006250,0.006250,1022.0,-10.0625,11.0,3.0,1986-01-09,1993-11-21,1990-01-05,1990,0.0,NaN,0.006250,0.006250
4,10001,7953,0.006211,0.006211,1022.0,10.1250,11.0,3.0,1986-01-09,1993-11-21,1990-01-08,1990,0.0,NaN,0.006211,0.006211


### Prune Stocks

In [18]:
# Prune stocks, map all equity to largest permno
crsp = util.maxCap(crsp)

crsp.head()

,permno,permco,ret,retx,shrcd,exchcd,namedt,nameendt,jdate,year,retadj,retxadj,me
2670,10001,7953,0.000000,0.000000,11.0,3.0,1986-01-09,1993-11-21,1990-01-02,1990,0.000000,0.000000,10347.750
8456,10001,7953,-0.012346,-0.012346,11.0,3.0,1986-01-09,1993-11-21,1990-01-03,1990,-0.012346,-0.012346,10220.000
14242,10001,7953,0.000000,0.000000,11.0,3.0,1986-01-09,1993-11-21,1990-01-04,1990,0.000000,0.000000,10220.000
20024,10001,7953,0.006250,0.006250,11.0,3.0,1986-01-09,1993-11-21,1990-01-05,1990,0.006250,0.006250,10283.875
25803,10001,7953,0.006211,0.006211,11.0,3.0,1986-01-09,1993-11-21,1990-01-08,1990,0.006211,0.006211,10347.750


In [19]:
# Trim daily returns to only those companies (permno) that have relevant market cap on dec 12 of that year
crsp = crsp[crsp['permno'].isin(permshort)]

crsp.head()

,permno,permco,ret,retx,shrcd,exchcd,namedt,nameendt,jdate,year,retadj,retxadj,me
2670,10001,7953,0.000000,0.000000,11.0,3.0,1986-01-09,1993-11-21,1990-01-02,1990,0.000000,0.000000,10347.750
8456,10001,7953,-0.012346,-0.012346,11.0,3.0,1986-01-09,1993-11-21,1990-01-03,1990,-0.012346,-0.012346,10220.000
14242,10001,7953,0.000000,0.000000,11.0,3.0,1986-01-09,1993-11-21,1990-01-04,1990,0.000000,0.000000,10220.000
20024,10001,7953,0.006250,0.006250,11.0,3.0,1986-01-09,1993-11-21,1990-01-05,1990,0.006250,0.006250,10283.875
25803,10001,7953,0.006211,0.006211,11.0,3.0,1986-01-09,1993-11-21,1990-01-08,1990,0.006211,0.006211,10347.750


### Merge in S&P 500

In [20]:
# Merge crsp and sp500
crsp = crsp.merge(crsp_sp,on=['jdate'])

### Top X by Market Cap

In [21]:
crsp = util.topX(crsp,'me',numstocks)

In [22]:
# Clear out any offending zero columns
crsp = crsp.dropna(how='all',axis=1)

# Clear out any remianing zero rows
crsp = crsp.dropna(how='any',axis=0)

In [23]:
crsp.head()

,permno,permco,ret,retx,shrcd,exchcd,namedt,nameendt,jdate,year,...,vwretd,vwretx,ewretd,ewretx,totval,totcnt,usdval,usdcnt,spindx,sprtrn
5,10016,1728,0.000000,0.000000,11.0,3.0,1986-12-29,1998-05-25,1990-01-02,1990,...,0.017527,0.017527,0.020005,0.020005,2.371903e+09,500.0,2.331046e+09,500.0,359.69,0.017799
6,10019,7971,0.000000,0.000000,11.0,3.0,1986-01-24,2002-06-18,1990-01-02,1990,...,0.017527,0.017527,0.020005,0.020005,2.371903e+09,500.0,2.331046e+09,500.0,359.69,0.017799
7,10020,7972,-0.018519,-0.018519,11.0,3.0,1986-01-27,1993-04-30,1990-01-02,1990,...,0.017527,0.017527,0.020005,0.020005,2.371903e+09,500.0,2.331046e+09,500.0,359.69,0.017799
8,10025,7975,0.008403,0.008403,11.0,3.0,1986-01-30,2004-06-09,1990-01-02,1990,...,0.017527,0.017527,0.020005,0.020005,2.371903e+09,500.0,2.331046e+09,500.0,359.69,0.017799
9,10026,7976,-0.050505,-0.050505,11.0,3.0,1986-02-04,2000-07-31,1990-01-02,1990,...,0.017527,0.017527,0.020005,0.020005,2.371903e+09,500.0,2.331046e+09,500.0,359.69,0.017799


### Loop Prep

In [24]:
# Do this stuff before the loop to save time
crsp['month'] = crsp['jdate'].dt.month
crsp['year'] = crsp['jdate'].dt.year
permlist['month'] = permlist['jdate'].dt.month
permlist['year'] = permlist['jdate'].dt.year
permlist['mktSD'] = 0
permlist['covmktstock'] = 0
permlist['beta'] = 0
crsp = crsp.sort_values(['permno','jdate'])
crsp = crsp.reset_index(drop=True)

#### *This is where the code differs for non-winsorized beta*

In [26]:
# Adjust returns to be used in loop
df = crsp
df.head()

,permno,permco,ret,retx,shrcd,exchcd,namedt,nameendt,jdate,year,...,vwretx,ewretd,ewretx,totval,totcnt,usdval,usdcnt,spindx,sprtrn,month
0,10001,7953,0.000850,0.000850,11.0,2.0,2009-12-18,2010-07-08,2010-06-08,2010,...,0.010778,0.009880,0.009652,9.847288e+09,500.0,9.742286e+09,500.0,1062.00,0.010976,6
1,10001,7953,0.016143,0.016143,11.0,2.0,2009-12-18,2010-07-08,2010-06-09,2010,...,-0.005761,-0.002144,-0.002249,9.790557e+09,500.0,9.847288e+09,500.0,1055.69,-0.005942,6
2,10001,7953,0.043257,0.043257,11.0,2.0,2009-12-18,2010-07-08,2010-06-14,2010,...,-0.001718,0.000477,0.000461,1.010139e+10,500.0,1.011878e+10,500.0,1089.63,-0.001805,6
3,10001,7953,0.030151,0.030151,11.0,2.0,2009-12-18,2010-07-08,2010-06-25,2010,...,0.002990,0.007330,0.007330,9.976495e+09,500.0,9.947060e+09,500.0,1076.76,0.002859,6
4,10001,7953,-0.035772,-0.035772,11.0,2.0,2009-12-18,2010-07-08,2010-06-28,2010,...,-0.002304,-0.003204,-0.003470,9.958139e+09,500.0,9.956838e+09,500.0,1074.57,-0.002034,6


to get a normal beta you would just not adjust the returns

#### Save Files "Pre For Loop"

##### can read back in "pre-loop" files if need be

## Beta Loop

In [27]:
# Initialize a dataframe
looplist = crsp['permno'].drop_duplicates().to_list()

# Loop to calculate betas directly using 
ll = len(looplist)

for i in range(ll):
    if(i%10==0):
        print(str(round(i/ll*100,2)) + '% complete')
        
    #Initialize loop dataframe
    loop_df = crsp[crsp['permno']==looplist[i]]
    year_loop_iter = permlist[permlist['permno']==looplist[i]]['year']

    for yiter in year_loop_iter:
        try:
            idx = permlist.loc[(permlist['permno']==looplist[i])
                &(permlist['year']==yiter)
                &(permlist['month']==12)].index[0]
        except:
            pass
        subloop_df = loop_df[(loop_df['year']>yiter-window)&(loop_df['year']<=yiter)]
        mktSD = subloop_df['sprtrn'].std()
        # Changed from wretadj to retadj to get normal beta
        covmktstock = subloop_df['sprtrn'].cov(subloop_df['retadj'])
        beta = covmktstock/mktSD**2
        permlist.loc[idx,'mktSD'] = mktSD
        permlist.loc[idx,'covmktstock'] = covmktstock
        permlist.loc[idx,'beta'] = beta

0.0% complete
0.09% complete
0.17% complete
0.26% complete
0.34% complete
0.43% complete
0.51% complete
0.6% complete
0.69% complete
0.77% complete


/Users/natejensen/opt/anaconda3/lib/python3.8/site-packages/pandas/core/nanops.py:1581: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/Users/natejensen/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


0.86% complete
0.94% complete
1.03% complete
1.12% complete
1.2% complete
1.29% complete
1.37% complete
1.46% complete
1.54% complete
1.63% complete
1.72% complete
1.8% complete
1.89% complete
1.97% complete
2.06% complete
2.14% complete
2.23% complete
2.32% complete
2.4% complete
2.49% complete
2.57% complete
2.66% complete
2.74% complete
2.83% complete
2.92% complete
3.0% complete
3.09% complete
3.17% complete
3.26% complete
3.35% complete
3.43% complete
3.52% complete
3.6% complete
3.69% complete
3.77% complete
3.86% complete
3.95% complete
4.03% complete
4.12% complete
4.2% complete
4.29% complete
4.37% complete
4.46% complete
4.55% complete
4.63% complete
4.72% complete
4.8% complete
4.89% complete
4.97% complete
5.06% complete
5.15% complete
5.23% complete
5.32% complete
5.4% complete
5.49% complete
5.58% complete
5.66% complete
5.75% complete
5.83% complete
5.92% complete
6.0% complete
6.09% complete
6.18% complete
6.26% complete
6.35% complete
6.43% complete
6.52% complete
6.6%

45.63% complete
45.72% complete
45.8% complete
45.89% complete
45.97% complete
46.06% complete
46.14% complete
46.23% complete
46.32% complete
46.4% complete
46.49% complete
46.57% complete
46.66% complete
46.75% complete
46.83% complete
46.92% complete
47.0% complete
47.09% complete
47.17% complete
47.26% complete
47.35% complete
47.43% complete
47.52% complete
47.6% complete
47.69% complete
47.77% complete
47.86% complete
47.95% complete
48.03% complete
48.12% complete
48.2% complete
48.29% complete
48.37% complete
48.46% complete
48.55% complete
48.63% complete
48.72% complete
48.8% complete
48.89% complete
48.98% complete
49.06% complete
49.15% complete
49.23% complete
49.32% complete
49.4% complete
49.49% complete
49.58% complete
49.66% complete
49.75% complete
49.83% complete
49.92% complete
50.0% complete
50.09% complete
50.18% complete
50.26% complete
50.35% complete
50.43% complete
50.52% complete
50.6% complete
50.69% complete
50.78% complete
50.86% complete
50.95% complete
5

89.89% complete
89.97% complete
90.06% complete
90.14% complete
90.23% complete
90.32% complete
90.4% complete
90.49% complete
90.57% complete
90.66% complete
90.75% complete
90.83% complete
90.92% complete
91.0% complete
91.09% complete
91.17% complete
91.26% complete
91.35% complete
91.43% complete
91.52% complete
91.6% complete
91.69% complete
91.77% complete
91.86% complete
91.95% complete
92.03% complete
92.12% complete
92.2% complete
92.29% complete
92.37% complete
92.46% complete
92.55% complete
92.63% complete
92.72% complete
92.8% complete
92.89% complete
92.98% complete
93.06% complete
93.15% complete
93.23% complete
93.32% complete
93.4% complete
93.49% complete
93.58% complete
93.66% complete
93.75% complete
93.83% complete
93.92% complete
94.0% complete
94.09% complete
94.18% complete
94.26% complete
94.35% complete
94.43% complete
94.52% complete
94.61% complete
94.69% complete
94.78% complete
94.86% complete
94.95% complete
95.03% complete
95.12% complete
95.21% complete

In [28]:
permlist.head(100)

,permno,jdate,month,year,mktSD,covmktstock,beta
0,10001,2011-12-31,12,2011,0.015311,0.000045,0.191367
1,10001,2012-12-31,12,2012,0.008042,0.000011,0.177603
2,10001,2014-12-31,12,2014,0.007163,0.000011,0.214663
3,10001,2016-12-31,12,2016,0.008193,0.000016,0.233114
4,10002,2002-12-31,12,2002,0.019160,0.000055,0.150805
...,...,...,...,...,...,...,...
306,10032,2005-12-31,12,2005,0.006479,0.000063,1.504223
307,10032,2006-12-31,12,2006,0.006322,0.000113,2.834012
308,10032,2007-12-31,12,2007,0.010070,0.000106,1.041029
309,10032,2008-12-31,12,2008,0.025808,0.000740,1.111356


### Export

In [29]:
permlist.to_csv("beta_"+str(window)+".csv.gz", 
           index=False, 
           compression="gzip")